In [4]:
!pip install yfinance

In [5]:
import yfinance as yf
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
import statsmodels as smt



start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
Visa = yf.download('V',start_date ,end_date,index=False)

[*********************100%***********************]  1 of 1 completed


In [6]:
Visa.reset_index(inplace=True)
Visa

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-03-01,149.460007,149.929993,148.380005,149.470001,146.724609,7016800
1,2019-03-04,150.070007,150.190002,146.949997,147.960007,145.242355,6979700
2,2019-03-05,148.000000,149.289993,147.830002,147.949997,145.232513,9678600
3,2019-03-06,148.000000,148.520004,147.419998,147.809998,145.095093,8186400
4,2019-03-07,147.259995,147.809998,146.039993,146.830002,144.133118,8761700
...,...,...,...,...,...,...,...
268,2020-03-24,144.839996,155.250000,143.580002,154.529999,152.619720,24488300
269,2020-03-25,159.320007,168.179993,153.580002,161.779999,159.780121,20619600
270,2020-03-26,162.080002,169.529999,161.789993,168.880005,166.792358,17062900
271,2020-03-27,161.080002,167.100006,158.149994,161.559998,159.562820,14950700


In [12]:
df = pd.read_csv('F-F_Research_Data_Factors_daily.CSV',skiprows=4)
df

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,19260701,0.10,-0.23,-0.28,0.009
1,19260702,0.45,-0.34,-0.03,0.009
2,19260706,0.17,0.29,-0.38,0.009
3,19260707,0.09,-0.59,0.00,0.009
4,19260708,0.21,-0.38,0.18,0.009
...,...,...,...,...,...
25164,20220126,-0.30,-1.08,0.09,0.000
25165,20220127,-0.78,-1.63,0.77,0.000
25166,20220128,2.45,-0.15,-2.07,0.000
25167,20220131,2.33,1.29,-2.86,0.000


In [13]:
df.rename(columns = {'Unnamed: 0':'Date'},inplace=True)

In [14]:
df=df[:-1]

In [15]:
df['Date']=pd.to_datetime(df['Date'])

<ipython-input-15-182e23b44254>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date']=pd.to_datetime(df['Date'])


In [16]:
df

,Date,Mkt-RF,SMB,HML,RF
0,1926-07-01,0.10,-0.23,-0.28,0.009
1,1926-07-02,0.45,-0.34,-0.03,0.009
2,1926-07-06,0.17,0.29,-0.38,0.009
3,1926-07-07,0.09,-0.59,0.00,0.009
4,1926-07-08,0.21,-0.38,0.18,0.009
...,...,...,...,...,...
25163,2022-01-25,-1.43,-0.60,2.79,0.000
25164,2022-01-26,-0.30,-1.08,0.09,0.000
25165,2022-01-27,-0.78,-1.63,0.77,0.000
25166,2022-01-28,2.45,-0.15,-2.07,0.000


In [17]:
Visa.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [18]:
df_final = Visa.merge(df, how='left', on='Date')

In [19]:
df_final

,Date,Open,High,Low,Close,Adj Close,Volume,Mkt-RF,SMB,HML,RF
0,2019-03-01,149.460007,149.929993,148.380005,149.470001,146.724609,7016800,0.72,0.28,-0.41,0.009
1,2019-03-04,150.070007,150.190002,146.949997,147.960007,145.242355,6979700,-0.52,-0.39,0.39,0.009
2,2019-03-05,148.000000,149.289993,147.830002,147.949997,145.232513,9678600,-0.17,-0.31,-0.23,0.009
3,2019-03-06,148.000000,148.520004,147.419998,147.809998,145.095093,8186400,-0.84,-1.27,0.05,0.009
4,2019-03-07,147.259995,147.809998,146.039993,146.830002,144.133118,8761700,-0.82,-0.07,-0.34,0.009
...,...,...,...,...,...,...,...,...,...,...,...
268,2020-03-24,144.839996,155.250000,143.580002,154.529999,152.619720,24488300,9.34,-1.27,2.43,0.006
269,2020-03-25,159.320007,168.179993,153.580002,161.779999,159.780121,20619600,1.18,-0.70,1.87,0.006
270,2020-03-26,162.080002,169.529999,161.789993,168.880005,166.792358,17062900,6.02,-0.75,1.17,0.006
271,2020-03-27,161.080002,167.100006,158.149994,161.559998,159.562820,14950700,-3.48,-0.84,-0.75,0.006


In [20]:
df_final.set_index(df_final['Date'], inplace=True) 

In [21]:
df_final['Daily_Rtn'] = df_final['Adj Close'].resample('D').last().pct_change().dropna()

In [22]:
df_final=df_final[1:]

In [23]:
X = df_final[['Mkt-RF', 'SMB', 'HML']]/100
y = df_final['Daily_Rtn'] - df_final['RF']
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     444.0
Date:                Fri, 22 Apr 2022   Prob (F-statistic):          1.26e-103
Time:                        15:20:29   Log-Likelihood:                 896.27
No. Observations:                 272   AIC:                            -1785.
Df Residuals:                     268   BIC:                            -1770.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0073      0.001    -13.215      0.0

In [24]:
X

,const,Mkt-RF,SMB,HML
Date,,,,
2019-03-04,1.0,-0.0052,-0.0039,0.0039
2019-03-05,1.0,-0.0017,-0.0031,-0.0023
2019-03-06,1.0,-0.0084,-0.0127,0.0005
2019-03-07,1.0,-0.0082,-0.0007,-0.0034
2019-03-08,1.0,-0.0022,0.0006,-0.0014
...,...,...,...,...
2020-03-24,1.0,0.0934,-0.0127,0.0243
2020-03-25,1.0,0.0118,-0.0070,0.0187
2020-03-26,1.0,0.0602,-0.0075,0.0117


In [25]:
y

Date
2019-03-04   -0.019102
2019-03-05   -0.009068
2019-03-06   -0.009946
2019-03-07   -0.015630
2019-03-08   -0.005459
                ...   
2020-03-24    0.132426
2020-03-25    0.040917
2020-03-26    0.037887
2020-03-27   -0.049345
2020-03-30    0.018821
Length: 272, dtype: float64

In [26]:
ff_model.params

const    -0.007345
Mkt-RF    1.109641
SMB      -0.362953
HML      -0.196039
dtype: float64